In [1]:
import face_rec1
import pandas as pd
import numpy as np
import cv2

/home/glccampos/Downloads/pass/envs/face_reco/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(
/home/glccampos/Downloads/pass/envs/face_reco/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:75: UserWarning: Duplicate provider 'CUDAExecutionProvider' encountered, ignoring.
  warnings.warn(f"Duplicate provider '{name}' encountered, ignoring.")


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: models/models/buffalo_sc/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: models/models/buffalo_sc/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: models/models/buffalo_sc/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: models/models/buffalo_sc/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: models/models/buffalo_sc/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


In [2]:
face_rec1.r.ping()

True

### Step1: Extract data from db

In [3]:
name = 'academy:register'
retrieved_dict = face_rec1.r.hgetall(name)
retrieved_series = pd.Series(retrieved_dict)
retrieved_series = retrieved_series.apply(lambda x: np.frombuffer(x,dtype=np.float32))
index = retrieved_series.index
index = list(map(lambda x: x.decode(), index))
retrieved_series.index = index
retrieved_df = retrieved_series.to_frame().reset_index()
retrieved_df.columns = ['name_role', 'facial_features']
retrieved_df[['Name', 'Role']] = retrieved_df['name_role'].apply(lambda x: x.split('@')).apply(pd.Series)
retrieved_df

,name_role,facial_features,Name,Role
0,Chris Evans@student,"[0.35691044, -0.41491055, 0.03921845, -0.15436...",Chris Evans,student
1,Barack Obama@teacher,"[-0.22177827, 1.5762169, -0.50307816, 0.136012...",Barack Obama,teacher
2,Angelina Jolie@student,"[0.19881934, 1.4748836, -0.0979588, -0.8805054...",Angelina Jolie,student
3,Gugubs@Student,"[-0.020469055, -1.1225978, -0.9341265, -1.7301...",Gugubs,Student
4,Scarlett Johansson@student,"[-0.13535985, -0.2821385, -0.4270084, -0.28251...",Scarlett Johansson,student
5,Morgan Freeman@teacher,"[0.7627893, 0.89061487, 0.46873555, 0.25898102...",Morgan Freeman,teacher


### Step2: get real time predictions

In [4]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if ret == False:
        break

    pred_frame = face_rec1.face_prediction(frame,
                                          retrieved_df,
                                          'facial_features',
                                          ['Name', 'Role'],
                                          thresh=0.5)
    
    cv2.imshow('frame', frame)
    cv2.imshow('frame', pred_frame)

    if cv2.waitKey(1) == 27:
        break

cv2.destroyAllWindows()
cap.release()

/home/glccampos/Downloads/pass/envs/face_reco/lib/python3.10/site-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4
/home/glccampos/Downloads/pass/envs/face_reco/lib/python3.10/site-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4
/home/glccampos/Downloads/pass/envs/face_reco/lib/python3.10/site-packages/insightface/utils/t